<a href="https://colab.research.google.com/github/mak109/cs6910_assignment2/blob/main/PART%20B/cs6910_assignment2_partB_question1_2_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading and Fine-tuning pretrained Models

In [ ]:
!pip install wget

In [ ]:
!pip install wandb -qqq
import wandb
wandb.login()

In [ ]:
from wandb.keras import WandbCallback

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
import numpy as np
import random
import wget
import os
from zipfile import ZipFile
from PIL import Image
from matplotlib import gridspec
plt.rcParams["figure.figsize"] = (20,10)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers,Sequential,regularizers,optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import * 
import random
random.seed(123)

In [ ]:
import wget
import os
from inspect import *
from zipfile import ZipFile
url='https://storage.googleapis.com/wandb_datasets/nature_12K.zip'
filename = os.path.basename(url)

if not os.path.exists(filename) and not os.path.exists("inaturalist_12K"):
  filename = wget.download(url)
  with ZipFile(filename, 'r') as zip:
    print('Extracting all the files now...')
    zip.extractall()
    print('Done!')
  os.remove(filename)
train_dir = 'inaturalist_12K/train'
test_dir = 'inaturalist_12K/val'

In [ ]:
image_size = (256,256)
num_classes = 10

In [ ]:
#Creating dictionary of models based on imagenet 
model_list = dict()
for key,value in getmembers(tf.keras.applications,isfunction):
    model_list[key] = value

In [ ]:
def CNN(config):
    base_model = model_list[config['model']](input_shape=image_size +(3,),include_top=False,weights='imagenet')
    base_model.trainable = True #this is important
    if(len(base_model.layers) > config['fine_tune_last']):
        for layer in base_model.layers[:-config['fine_tune_last']]:
            layer.trainable = False    
    global_average_layer = layers.GlobalAveragePooling2D()
    prediction_layer = layers.Dense(num_classes,activation='softmax')
    inputs = layers.Input((image_size[0],image_size[1],3))
    input_rescale=layers.Rescaling(1./255)(inputs)
    x = base_model(input_rescale)    
    x = global_average_layer(x)
    x = layers.Dropout(config['dropout'])(x)
    outputs = prediction_layer(x)
    model = keras.Model(inputs,outputs)
    return model

In [ ]:
def train():
    config_ = {
    "model": 'InceptionV3',
    "learning_rate": 1e-4,
    "data_augment": "True",
    "dropout":0.2,
    "batch_size":32,
    "fine_tune_last":20,
    "epochs":3
    }
    val_generator = ImageDataGenerator(dtype=tf.float32,validation_split=0.1,data_format='channels_last').flow_from_directory(
        train_dir,
        target_size = image_size,
        batch_size = config['batch_size'],
        color_mode = 'rgb',
        class_mode = 'sparse',
        shuffle=True,
        subset='validation',
        seed=123
    
    )
    #Data Augmentation
    if config["data_augment"] == 'True':
        data_generator = ImageDataGenerator(
        rotation_range=50, #random rotation between -50(clockwise) to 50(anti-clockwise) degree
        brightness_range=(0.2,0.8), 
        zoom_range=0.3, #zoom in range from [0.7,1.3]
        horizontal_flip=True,
        vertical_flip=True,
        width_shift_range=0.1, #Horizontal Shifting as a ratio of width
        height_shift_range=0.2,#Vertical Shifting as a ratio of height
        data_format='channels_last',
        validation_split=0.1,
        dtype=tf.float32
        )
    else:
        data_generator = ImageDataGenerator(
            data_format='channels_last',
            validation_split=0.1,
            dtype=tf.float32
        )
    #Train set creation after conditional augmentation
    train_generator = data_generator.flow_from_directory(
    train_dir,
    target_size = image_size,
    batch_size = config['batch_size'],
    color_mode = 'rgb',
    class_mode = 'sparse',
    shuffle=True,
    subset='training',
    seed=123
    )
    
    model = CNN(config)
    
    model.compile(
    optimizer=optimizers.Adam(learning_rate=config["learning_rate"]),
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
    )
    
    history = model.fit(train_generator,
                epochs=config['epochs'],
                validation_data=val_generator)
    return history,model

In [ ]:
config = {
    "model": 'InceptionV3',
    "learning_rate": 1e-4,
    "data_augment": "False",
    "dropout":0.2,
    "batch_size":32,
    "fine_tune_last":20,
    "epochs":3
    }
model = CNN(config)

In [ ]:
history,model = train()

In [ ]:
val_generator = ImageDataGenerator(dtype=tf.float32,validation_split=0.1,data_format='channels_last').flow_from_directory(
        train_dir,
        target_size = image_size,
        batch_size = config['batch_size'],
        color_mode = 'rgb',
        class_mode = 'sparse',
        shuffle=True,
        subset='validation',
        seed=123
    
    )
    #Data Augmentation
if config["data_augment"] == 'True':
    data_generator = ImageDataGenerator(
        rotation_range=50, #random rotation between -50(clockwise) to 50(anti-clockwise) degree
        brightness_range=(0.2,0.8), 
        zoom_range=0.3, #zoom in range from [0.7,1.3]
        horizontal_flip=True,
        vertical_flip=True,
        width_shift_range=0.1, #Horizontal Shifting as a ratio of width
        height_shift_range=0.2,#Vertical Shifting as a ratio of height
        data_format='channels_last',
        validation_split=0.1,
        dtype=tf.float32
        )
else:
    data_generator = ImageDataGenerator(
            data_format='channels_last',
            validation_split=0.1,
            dtype=tf.float32
        )
    #Train set creation after conditional augmentation
train_generator = data_generator.flow_from_directory(
    train_dir,
    target_size = image_size,
    batch_size = config['batch_size'],
    color_mode = 'rgb',
    class_mode = 'sparse',
    shuffle=True,
    subset='training',
    seed=123
    )
fine_tune_epochs = 5
total_epochs =  config['epochs'] + fine_tune_epochs

history_fine = model.fit(train_generator,
                         epochs=total_epochs,
                         initial_epoch=history.epoch[-1],
                         validation_data=val_generator)

In [ ]:
test_generator = ImageDataGenerator(dtype=tf.float32,validation_split=0.0,data_format='channels_last').flow_from_directory(
        'inaturalist_12K/val',
        target_size = image_size,
        batch_size = 200,
        color_mode = 'rgb',
        class_mode = 'sparse',
        shuffle=False,
        seed=123
        )
loss0, accuracy0 = model.evaluate(test_generator)